In [41]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count


spark = SparkSession.builder.appName("Assignment").getOrCreate()

df = spark.read.csv('Electric_Vehicle_Population_Data.csv',header=True,inferSchema=True)

In [42]:

# Περιγραφή columns

# VIN (1-10): Μοναδικός αριθμός οχήματος.
# County: Επαρχία όπου είναι καταχωρημένο το όχημα.
# City: Πόλη όπου είναι καταχωρημένο το όχημα.
# State: Πολιτεία καταχώρησης.
# Postal Code: Ταχυδρομικός κώδικας καταχώρησης.
# Model Year: Έτος κατασκευής του οχήματος.
# Make: Κατασκευαστής του οχήματος (π.χ. Tesla, BMW).
# Model: Μοντέλο του οχήματος (π.χ. Model 3, X5).
# Electric Vehicle Type: Τύπος ηλεκτρικού οχήματος (BEV ή PHEV).
# CAFV Eligibility: Επιλεξιμότητα για κίνητρα καθαρών καυσίμων.
# Electric Range: Απόσταση που μπορεί να διανύσει το όχημα με πλήρη φόρτιση (σε μίλια).
# Base MSRP: Βασική προτεινόμενη τιμή λιανικής (χωρίς φόρους).
# Legislative District: Νομοθετική περιφέρεια καταχώρησης.
# DOL Vehicle ID: Μοναδικός αριθμός οχήματος από το Υπουργείο Συγκοινωνιών.
# Vehicle Location: Γεωγραφικές συντεταγμένες καταχώρησης.
# Electric Utility: Πάροχος ηλεκτρικού ρεύματος.
# 2020 Census Tract: Κωδικός απογραφικής περιοχής από την απογραφή του 2020.


# Εφόσον θα ασχοληθούμε με στατιστική, αποφασίζουμε οτι δεν χρειαζόμαστε για τον σκοπό μας τα columns : 

#  Legislative District
# DOL Vehicle ID (έχουμε ήδη ἐνα identification number)
# 2020 Census Tract


columns_to_drop = ["Legislative District", "DOL Vehicle ID", "2020 Census Tract"]
df = df.drop(*columns_to_drop)



print("Remaining columns:", df.columns)

Remaining columns: ['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year', 'Make', 'Model', 'Electric Vehicle Type', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range', 'Base MSRP', 'Vehicle Location', 'Electric Utility']


In [43]:
#Για λόγους ευκολίας κατανόησης θα μετονομάσουμε τα παρακάτω columns:

# VIN (1-10): ID
# CAFV Eligibility: Eligible for clean energy tax reductions
# Base MSRP: Suggested price


df= (df
    .withColumnRenamed("VIN (1-10)", "ID")
    .withColumnRenamed("Clean Alternative Fuel Vehicle (CAFV) Eligibility", "Eligible for clean energy tax reductions")
    .withColumnRenamed("Base MSRP", "Suggested price")
)


# Ελέγχουμε τις αλλαγές
print("Updated columns:", df.columns)


Updated columns: ['ID', 'County', 'City', 'State', 'Postal Code', 'Model Year', 'Make', 'Model', 'Electric Vehicle Type', 'Eligible for clean energy tax reductions', 'Electric Range', 'Suggested price', 'Vehicle Location', 'Electric Utility']


In [44]:
#Ελέγχουμε ποιές εγγραφές περιέχουν null τιμές και μπορεί να μας δημιουργήσουν πρόβλημα κατα την επεξεργασία και στατιστική

null_counts = df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns
])
null_counts.show()

+---+------+----+-----+-----------+----------+----+-----+---------------------+----------------------------------------+--------------+---------------+----------------+----------------+
| ID|County|City|State|Postal Code|Model Year|Make|Model|Electric Vehicle Type|Eligible for clean energy tax reductions|Electric Range|Suggested price|Vehicle Location|Electric Utility|
+---+------+----+-----+-----------+----------+----+-----+---------------------+----------------------------------------+--------------+---------------+----------------+----------------+
|  0|     3|   3|    0|          3|         0|   0|    0|                    0|                                       0|             0|              0|               9|               3|
+---+------+----+-----+-----------+----------+----+-----+---------------------+----------------------------------------+--------------+---------------+----------------+----------------+



In [45]:
# Εφόσον παρατηρούμε ίδιο αριθμό null values σε κάθε column,
# υποψιαζόμαστε ότι μπορεί να πρόκειται για τις ίδιες εγγραφές, συνεπώς εφόσον πρόκειται για 3, 
# μπορούμε να τις αφαιρεσουμε αφόυ ειναι ένα πολύ μικρο ποσοστό αναφορικά με όλα τα rows μας (200000)


df = df.dropna(subset=["County"])

# Έπειτα ξανα ελέγχουμε για null τιμές

null_counts = df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns
])
null_counts.show()

+---+------+----+-----+-----------+----------+----+-----+---------------------+----------------------------------------+--------------+---------------+----------------+----------------+
| ID|County|City|State|Postal Code|Model Year|Make|Model|Electric Vehicle Type|Eligible for clean energy tax reductions|Electric Range|Suggested price|Vehicle Location|Electric Utility|
+---+------+----+-----+-----------+----------+----+-----+---------------------+----------------------------------------+--------------+---------------+----------------+----------------+
|  0|     0|   0|    0|          0|         0|   0|    0|                    0|                                       0|             0|              0|               6|               0|
+---+------+----+-----+-----------+----------+----+-----+---------------------+----------------------------------------+--------------+---------------+----------------+----------------+



In [48]:
# Ελέγχουμε τις μοναδικές τιμές για κάθε column. 

for column in df.columns:
    if column != 'ID':
        unique_values = df.select(column).distinct().collect()
        print(f"Unique values in column '{column}':")
        print([row[column] for row in unique_values])
        print(f"Count of unique values: {len(unique_values)}\n")


Unique values in column 'County':
['Thurston', 'Kittitas', 'Snohomish', 'Kitsap', 'Okanogan', 'Grays Harbor', 'King', 'Whatcom', 'Pierce', 'Mason', 'Skagit', 'Pend Oreille', 'Benton', 'Lewis', 'Yakima', 'Whitman', 'Grant', 'Cowlitz', 'Chelan', 'Clark', 'Spokane', 'Island', 'Stevens', 'Douglas', 'Clallam', 'Jefferson', 'Walla Walla', 'Klickitat', 'Franklin', 'Lincoln', 'Pacific', 'San Juan', 'Sarasota', 'Skamania', 'Monterey', 'Columbia', 'Asotin', 'Wahkiakum', 'Adams', 'Wasco', 'Richland', 'Orange', 'Lake', 'San Diego', 'District of Columbia', 'Kern', 'Wake', 'Lee', 'Ferry', 'DeKalb', 'San Francisco', 'Oldham', 'Anne Arundel', 'Churchill', 'Riverside', 'Polk', 'New York', 'Kings', 'Leavenworth', 'Los Angeles', 'Sacramento', 'Montgomery', 'Santa Clara', 'Madison', 'San Mateo', 'Garfield', 'Allegheny', 'Fairfax', 'Macomb', "Prince George's", 'Anchorage', 'Autauga', 'Ventura', 'Alexandria', 'Palm Beach', 'Allen', 'Cook', 'Saratoga', 'Hudson', 'Harnett', 'El Paso', 'Geary', 'Stafford', 'Ne